In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tflite_runtime.interpreter as tflite
from FaceRecognizer import get_quant_int8_output

In [3]:
REC_MODEL_PATH = "/home/yzy/PycharmProjects/Mobilefacenet-TF2/model/mbfacenet_mask_quant.tflite"
interpreter_rec = tf.compat.v1.lite.Interpreter(model_path=REC_MODEL_PATH)
interpreter_rec.allocate_tensors()

In [4]:
input_index = interpreter_rec.get_input_details()[0]['index']
embeds_output_index = interpreter_rec.get_output_details()[1]['index']
mask_output_index = interpreter_rec.get_output_details()[0]['index']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

data_path = "/home/yzy/Downloads/Face-Mask-Detection-master/dataset"
def load_mask_dataset():
    X, y = [], []
    for root, dirs, filenames in os.walk(data_path):
        for file in filenames:
            if os.path.splitext(file)[-1] == '.png' or os.path.splitext(file)[-1] == '.jpg':
                X.append(os.path.join(root, file))
                y.append(os.path.split(root)[-1])
    le = LabelEncoder()
    y = le.fit_transform(y)
    print(le.classes_)
    trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1)
    return trainX, testX, trainy, testy

def preprocess(x, y):
    # x: directory，y：label
    x = tf.io.read_file(x)
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    x = tf.image.resize(x, [112, 96])

    x = tf.image.random_flip_left_right(x)

    # x: [0,255]=> -1~1
    # x = (tf.cast(x, dtype=tf.float32) - 127.5) / 128.0
    x = tf.cast(x, dtype=tf.uint8)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=2)

    return x, y

# get data slices
train_image, val_image, train_label, val_label = load_mask_dataset()

['with_mask' 'without_mask']


In [6]:
# construct input pipeline
batchsz = 8
db_train = tf.data.Dataset.from_tensor_slices((train_image, train_label))     # construct train dataset
db_train = db_train.shuffle(1000).map(preprocess).batch(batchsz)
db_val = tf.data.Dataset.from_tensor_slices((val_image, val_label))
db_val = db_val.shuffle(1000).map(preprocess).batch(batchsz)

In [7]:
test_data = next(iter(db_val))

In [8]:
scale, zero = interpreter_rec.get_input_details()[0]['quantization']
for d in test_data[0]:

    print(d.shape)
    # d = (d / scale) + zero
    # d = tf.uint8(d)
    # print(d.shape)
    d = tf.expand_dims(d, axis=0)
    d = tf.convert_to_tensor(d, dtype=tf.uint8)
    interpreter_rec.set_tensor(input_index, d)
    interpreter_rec.invoke()
    print(get_quant_int8_output(interpreter_rec, mask_output_index))

(112, 96, 3)
[[0.         0.99609375]]
(112, 96, 3)
[[0.99609375 0.00390625]]
(112, 96, 3)
[[0.99609375 0.        ]]
(112, 96, 3)
[[0.99609375 0.        ]]
(112, 96, 3)
[[0.         0.99609375]]
(112, 96, 3)
[[0.         0.99609375]]
(112, 96, 3)
[[0.98828125 0.01171875]]
(112, 96, 3)
[[0.95703125 0.04296875]]


In [9]:
test_data[1]

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)>

In [12]:
# test edge_tpu
TPU_MODEL = "/home/yzy/PycharmProjects/Mobilefacenet-TF2/model/model_with_mask_clf_quant_edgetpu.tflite"
interpreter_tpu = tflite.Interpreter(model_path=TPU_MODEL,
                                             experimental_delegates=[tflite.load_delegate("libedgetpu.so.1")])
interpreter_tpu.allocate_tensors()

In [13]:
input_index_tpu = interpreter_tpu.get_input_details()[0]['index']
embeds_output_index_tpu = interpreter_tpu.get_output_details()[1]['index']
mask_output_index_tpu = interpreter_tpu.get_output_details()[0]['index']


In [19]:
for d in test_data[0]:

    # print(d.shape)
    # d = (d / scale) + zero
    # d = tf.uint8(d)
    # print(d.shape)
    d = tf.expand_dims(d, axis=0)
    d = tf.convert_to_tensor(d, dtype=tf.uint8)
    interpreter_tpu.set_tensor(input_index_tpu , d)
    interpreter_tpu.invoke()
    print(get_quant_int8_output(interpreter_tpu, mask_output_index_tpu))

[[0.         0.99609375]]
[[0.99609375 0.00390625]]
[[0.99609375 0.        ]]
[[0.99609375 0.        ]]
[[0.         0.99609375]]
[[0.         0.99609375]]
[[0.99609375 0.00390625]]
[[0.9921875 0.0078125]]
